In [3]:
import requests
import json
import os
import csv
import pandas as pd
MY_API_KEY='9eab72c1b16e7eabbcc3b4cceb7851e7'
path_combination_file="combination.json"
queries_csv="queries.csv"

def generate_combination(path_combination_file,queries_csv):
    data = open(path_combination_file,) 
    
    data=json.load(data)
    combinations=[]
    for filter in data['filter']:
            for iot in data['iot']:
                for logic1 in data['logic1']:
                    for logic2 in data['logic2']:                        
                        combinations.append({"filter":filter,"management":"change management","soft":"soft skills","iot":iot,"logic1":logic1,"logic2":logic2,"result":""})
    for filter in data['filter']:
        for logic1 in data['logic1']:            
                combinations.append({"filter":filter,"management":"change management","soft":"soft skills","iot":'',"logic1":logic1,"logic2":"","result":""})
    df = pd.DataFrame(combinations, columns=['filter', 'management','soft','iot','logic1','logic2','result'])
    df.to_csv(queries_csv, index=False)  

def get_queries(path_file):
    df=pd.read_csv(path_file)
    return df.to_dict(orient='records')

def get_chunk(key,start,count='25',date='2010-2020'):

    resp = requests.get(f"https://api.elsevier.com/content/search/scopus?query={key}&start={start}&count={count}&date={date}",
                    headers={'Accept':'application/json',
                             'X-ELS-APIKey': MY_API_KEY})
    data=json.loads(resp.text.encode('utf-8'))
    return data

def compose_query(fields_query):
    if('management' in fields_query and 'iot' in fields_query):
        query=f"{fields_query['filter']}%28{fields_query['management']}+{fields_query['logic1']}+{fields_query['soft']}+{fields_query['logic2']}+{fields_query['iot']}%29"
    else:
        query=f"{fields_query['filter']}%28{fields_query['management']}+{fields_query['logic1']}+{fields_query['soft']}%29"
    return query

def write_result(path_file,fields_query,result):
    df=pd.read_csv(path_file)
    df.loc[(df['filter'] == fields_query['filter']) & (df['management']==fields_query['management']) & 
    (df['soft']==fields_query['soft']) & (df['iot']==fields_query['iot']) & (df['logic1']==fields_query['logic1']) & (df['logic2']==fields_query['logic2']),'result']=result

    df.to_csv(path_file,index=False,)

def main():
    # generate_combination(path_combination_file,queries_csv)
    queries=get_queries(queries_csv)
    composed_query=compose_query(queries[0])
    print(composed_query)
    # for query in queries:
    #         composed_query=compose_query(query)
    #         data=get_chunk(composed_query,start=0,count=25)
    #         total_results=int(data['search-results']['opensearch:totalResults'])
    #         write_result(queries_csv,query,total_results)

if __name__ == "__main__":
    main()

KeyError: 'filter'